In [ ]:
from sqlalchemy import create_engine
from collections import Counter
import io
import pandas as pd
import sqlite3 as sql
import re
import numpy as np
import seaborn as sns
from datetime import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from jupyterthemes import jtplot
jtplot.style()
from shapely.geometry import Point
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from skimpy import clean_columns
import cmd
import geopandas as gpd
# !pip install skimpy
# !pip install openpyxl
pd.set_option('display.max_rows', 100)
# pd.set_option('max_columns', None)
pd.options.display.max_colwidth = 3000

%matplotlib inline

In [ ]:
us = pd.read_csv('US_data.csv')
homes = pd.read_csv('../data/City_MedianRentalPrice_AllHomes.csv')
# caperson = pd.read_csv('crimes_against_persons.csv')
# caproperty = pd.read_csv('crimes_against_property.csv')
# casociety = pd.read_csv('crimes_against_society.csv')
# ids = pd.read_csv('incidents_with_drugs_seized.csv')
# ouda = pd.read_csv('offenders_using_drugs_alcohol.csv')
# nmv = pd.read_csv('negligent_manslaughter_victims.csv')
# mnmaav = pd.read_csv('murder_nonnegligent_manslaughter_aggravated_assault_victims.csv')
# ijkjhaah = pd.read_csv('individuals_justifiably_killed_justified_homicide_agggravated_assault_homicide.csv')

In [ ]:
us.head(100)

In [ ]:
us_list = us.columns.values.tolist()

In [ ]:
cli = cmd.Cmd()
cli.columnize(us_list, displaywidth = 200)

In [ ]:
us

In [ ]:
us[(us.state == '0') | (us.state == 'None')] = None
us['state_city'] = us.state.fillna(method = 'ffill', axis = 0)

In [ ]:
us

In [ ]:
us['state_city'] = us['state_city'].fillna(method = 'backfill')
us['state'] = us['state_city']

In [ ]:
us.head(100)

In [ ]:
us[(us.agency_type == '0') | (us.agency_type == 'None')] = None
us['agency_type_new'] = us.agency_type.fillna(method = 'ffill', axis = 0)

In [ ]:
us['agency_type_new'] = us['agency_type_new'].fillna(method = 'backfill')
us['agency_type'] = us['agency_type_new']

In [ ]:
us = us.drop(['state_city', 'agency_type_new'], axis = 1)
us

In [ ]:
us

In [ ]:
us.head(100)

In [ ]:
us.tail(100)

In [ ]:
us.value_counts('state')

In [ ]:
west1 = us[us['state'].isin(['WASHINGTON', 'OREGON', 'NEVADA', 'WYOMING', 'MONTANA', 'IDAHO', 'UTAH', 'NEW MEXICO', 'ARIZONA', 'COLORADO'])]
west1

In [ ]:
west = us[us['state'].isin(['WASHINGTON', 'OREGON', 'NEVADA', 'WYOMING', 'MONTANA', 'IDAHO', 'UTAH', 'NEW MEXICO', 'ARIZONA', 'COLORADO'])].groupby('state').sum()

In [ ]:
midwest = us[us['state'].isin(['NORTH DAKOTA', 'SOUTH DAKOTA', 'NEBRASKA', 'KANSAS', 'MISSOURI', 'MINNESOTA', 'IOWA', 'WISCONSIN', 'ILLINOIS', 'INDIANA', 'MICHIGAN', 'OHIO'])].groupby('state').sum()

In [ ]:
north = us[us['state'].isin(['PENNSYLVANIA', 'NEW YORK', 'DISTRICT OF COLUMBIA', 'VERMONT', 'CONNECTICUT', 'RHODE ISLAND', 'MASSACHUSETTS', 'NEW HAMPSHIRE', 'MAINE'])].groupby('state').sum()

In [ ]:
south = us[us['state'].isin(['TEXAS', 'OKLAHOMA', 'ARKANSAS', 'LOUISIANA', 'MISSISSIPPI', 'TENNESSEE', 'KENTUCKY', 'GEORGIA', 'SOUTH CAROLINA', 'NORTH CAROLINA', 'WEST VIRGINIA', 'VIRGINIA', 'MARYLAND', 'DELAWARE'])].groupby('state').sum()

In [ ]:
federal = us[us['state'].isin(['FEDERAL5'])].groupby('state').sum()

In [ ]:
west

In [ ]:
west = west.reset_index()
west

In [ ]:
midwest = midwest.reset_index()
midwest

In [ ]:
north = north.reset_index()
north

In [ ]:
north.dtypes

In [ ]:
south = south.reset_index()
south

In [ ]:
south.dtypes

In [ ]:
federal = federal.reset_index()
federal

In [ ]:
homes.head(100)

In [ ]:
print(homes.columns.tolist())

In [ ]:
homes = homes.drop(['Unnamed: 0', 'Metro'], axis = 1)
homes

In [ ]:
homes['mean'] = homes.filter(regex = "[^2010-01]").mean(1)

In [ ]:
homes

In [ ]:
test = {}

In [ ]:
test["mean"] = homes.loc[:,[c for c in homes.columns if c!= "2010-01"]].mean(axis = 1)

In [ ]:
test

In [ ]:
us = us.reset_index()
us

In [ ]:
homes = homes.reset_index()
homes

In [ ]:
homes.loc[homes['State']=='VT']

In [ ]:
us.to_csv('us_clean.csv', index = False)

In [ ]:
west.to_csv('west.csv', index = False)

In [ ]:
midwest.to_csv('midwest.csv', index = False)

In [ ]:
north.to_csv('north.csv', index = False)

In [ ]:
south.to_csv('south.csv', index = False)

In [ ]:
federal.to_csv('federal.csv', index = False)

In [ ]:
homes.to_csv('homes.csv', index = False)

In [ ]:
us

In [ ]:
us.columns.to_list()

In [ ]:
us_test = us.sort_values(by = ["state", "agency_name"])[['index', 'state', 'agency_type', 'agency_name', 'population_1', 'total_offenses', 'crimes_against_persons', 'crimes_against_property',
 'crimes_against_society', 'assault_offenses', 'aggravated_assault', 'simple_assault', 'intimidation', 'homicide_offenses', 'murder_and_nonnegligent_manslaughter', 'negligent_man_slaughter',
 'justifiable_homicide', 'human_trafficking_offenses', 'commercial_sex_acts', 'involuntary_servitude', 'kidnapping_abduction', 'sex_offenses', 'rape', 'sodomy', 'sexual_assault_with_an_object',
 'fondling', 'incest', 'statutory_rape', 'arson', 'bribery', 'burglary_breaking_&_entering', 'counter_feiting_forgery', 'destruction_damage_vandalism_of_property', 'embezzle_ment', 'extortion_blackmail',
 'fraud_offenses', 'false_pretenses_swindle_confidence_game', 'credit_card_automated_teller_machine_fraud', 'imper_sonation', 'welfare_fraud', 'wire_fraud', 'identity_theft', 'hacking_computer_invasion',
 'larceny_theft_offenses', 'pocket_picking', 'purse_snatching', 'shop_lifting', 'theft_from_building', 'theft_from_coin_op_erated_machine_or_device', 'theft_from_motor_vehicle',
 'theft_of_motor_vehicle_parts_or_acces_sories', 'all_other_larceny', 'motor_vehicle_theft', 'robbery', 'stolen_property_offenses', 'animal_cruelty', 'drug_narcotic_offenses', 'drug_narcotic_violations',
 'drug_equipment_violations', 'gambling_offenses', 'betting_wagering', 'operating_promoting_assisting_gambling', 'gambling_equipment_violations', 'sports_tampering', 'por_nography_obscene_material',
 'pros_titution_offenses', 'pros_titution', 'assisting_or_promoting_prostitution', 'purchasing_prostitution', 'weapon_law_violations']]

In [ ]:
us_test